In [1]:
# import
import glob
import numpy as np


import numpy as np
import pandas as pd
import sys
# import seaborn as sns
from numpy import random

from matplotlib import pyplot as plt
import sys
import time
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
# import original_data
import optuna
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, random_split

from torch.cuda.amp import GradScaler, autocast
# config
LEARNING_RATE = 0.0001
BATCH_SIZE =  256
EPOCHS = 400
N_FRAME = 12
# TIME_LENGTH = 40
BEFORE_LENGTH = 10 # 20 30
AFTER_LENGTH = 10 # 20 30 
N_SAMPLE = 10000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PATH = './data/timedt.data.400'

def get_original_data(
   path:str
):
    """
    input temperature : 温度
    
    return t, msd, csp, xyz, r_, v_xyz, v_, angle, g

    t       时间序列(单位:ps)
    msd     单He的msd(均方位移)
    csp     CSP(中心对称参数)
    xyz     单He的xyz坐标
    r_      单He离原点距离
    v_xyz   单He的沿xyz坐标的速度分量
    v_      单He的速度大小
    """
    with open(path, 'r', encoding='utf-8') as fin:
        t = []      # 时间序列(单位:ps)
        msd = []    # 单He的msd(均方位移)
        csp = []    # CSP(中心对称参数)
        xyz = []    # 单He的xyz坐标
        r_ = []     # 单He离原点距离
        v_xyz = []  # 单He的沿xyz坐标的速度分量
        v_ = []     # 单He的速度大小
        for i, line in enumerate(fin.readlines()[1:]):
            data = list(map(float, line.strip().split(' ')))
            t.append(data[0])
            msd.append(data[1])
            csp.append(data[2:8])
            xyz.append(data[8:11])
            r_.append(data[11])
            v_xyz.append(data[12:15])
            v_.append(data[15])

    # with open(G_PATH.format(Temperature=temperature), 'r', encoding='utf-8') as fin:
    #     g = []      # g参数
    #     for i, line in enumerate(fin.readlines()[1:]):
    #         data = list(map(float, line.strip().split(' ')))
    #         g.append(data[1:7])
    # indices_to_remove = np.arange(1001, len(t) - 1, 1001)
    t = np.array(t)
    # t = np.delete(t, indices_to_remove)
    t = t.reshape(-1, 1)
    msd = np.array(msd)
    # msd = np.delete(msd, indices_to_remove)
    msd = msd.reshape(-1, 1)
    csp = np.array(csp)
    # csp = np.delete(csp, indices_to_remove, axis=0)
    xyz = np.array(xyz)
    # xyz = np.delete(xyz, indices_to_remove, axis=0)
    r_ = np.sqrt(np.array(r_))
   #  r_ = np.delete(r_, indices_to_remove)
    r_ = r_.reshape(-1, 1)
    v_xyz = np.array(v_xyz)
    # v_xyz = np.delete(v_xyz, indices_to_remove, axis=0)
    v_ = np.sqrt(np.array(v_))
    # v_ = np.delete(v_, indices_to_remove)
    v_ = v_.reshape(-1, 1)
    angle = np.arccos(v_xyz / v_.reshape(len(t), 1))
    # g = np.array(g)

    return t, msd, csp, xyz, r_, v_xyz, v_, angle# , g
def get_iter(path, batch_size):
    '''
    这个index有没有用另外说
    '''
    data = get_original_data(PATH)
    # index = np.where(label[TIME_LENGTH//2 : -TIME_LENGTH//2])[0] + TIME_LENGTH//2
    # data_num = int(len(data[0])/TIME_LENGTH)
    data_num = int(len(data[0])/(BEFORE_LENGTH + AFTER_LENGTH))
    # diff_xyz = np.diff(data[3], axis=0)
    # diff_xyz = np.vstack(([0, 0, 0], diff_xyz))
    # y = diff_xyz[:, 0].reshape(-1, 1)
    y = data[3][:,2].reshape(-1,1)
    data = np.hstack(data[1:])
    
    # data = data[3]
    INPUT_SIZE = data.shape[-1]
    scaler = StandardScaler()
    
    data = scaler.fit_transform(data)
    # y = scaler.fit_transform(y)
    
    data_X = torch.from_numpy(data)
    # print(data_X.shape)
    data_Y = torch.from_numpy(y)
    data_Y = data_Y.squeeze(1)

    X = torch.zeros(data_num, BEFORE_LENGTH, INPUT_SIZE)
    # Y = torch.zeros(data_num, TIME_LENGTH//2)
    Y = torch.zeros(data_num, 1)
    i=0
        # for i, idx in enumerate(sample_indices):
    # for idx in range(TIME_LENGTH//2, len(data)-TIME_LENGTH//2, TIME_LENGTH):
    #     X[i, :, :] = data_X[idx-TIME_LENGTH//2 : idx]
    #     Y[i, :]    = data_Y[idx+TIME_LENGTH//2]# -data_Y[idx]
    #     i+=1
    
    for idx in range(BEFORE_LENGTH, len(data)-AFTER_LENGTH, (BEFORE_LENGTH+AFTER_LENGTH)):
        #print(idx-BEFORE_LENGTH)
        X[i, :, :] = data_X[idx-BEFORE_LENGTH : idx]
        
        Y[i, :]    = data_Y[idx+AFTER_LENGTH] -data_Y[idx]
        i+=1
    Y = F.normalize(Y, p=2, dim=1)
    X = X.float().unsqueeze(1) # [Batch_size, C, H, W]
    shuffled_index = np.random.permutation(range(data_num))
    X = X[shuffled_index]
    Y = Y[shuffled_index]
    Y = Y.view(Y.size(0), -1)
    Train_X, Test_X = X[:int(data_num*0.8)], X[int(data_num*0.8):]
    Train_Y, Test_Y = Y[:int(data_num*0.8)], Y[int(data_num*0.8):]


    Train_generator = DataLoader(
        torch.utils.data.TensorDataset(Train_X, Train_Y), 
        batch_size, 
        shuffle=True
    )
    Test_generator = DataLoader(
        torch.utils.data.TensorDataset(Test_X, Test_Y), 
        batch_size, 
        shuffle=True
    )
    return Train_generator, Test_generator

class CNN(nn.Module):
    def __init__(
        self, 
        Channel_in, 
        Height_in, 
        Width_in,
        Embedding_list,
        Output_size, 
        Filter_num, 
        Kernel_list, 
        dropout = 0.5, 
    ):
        super(CNN, self).__init__()
        self.Embedding_list = Embedding_list
        self.emb = nn.ModuleList([
            nn.Sequential( nn.Linear(emb, Width_in),)
         for emb in Embedding_list])
        
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(
                    Channel_in, 
                    Filter_num, 
                    kernel_size=(kernel, Width_in), 
                    padding=((kernel - 1) // 2, 0), 
                ),
                nn.ReLU(),
                nn.MaxPool2d(
                    kernel_size=((Height_in+3)//4, 1), 
                    stride=(Height_in+3)//4, 
                    padding=((Height_in-Height_in//4*4+1)//2, 0), 
                ), 
            )
            for kernel in Kernel_list
        ])
        # print(Kernel_list)
        self.fc = nn.Sequential(
            nn.Dropout(dropout), 
            nn.Linear(Filter_num * len(Kernel_list) * 4, 64),
            nn.ReLU(), 
            nn.Linear(64, 16), 
            nn.Linear(16, Output_size)
        )
        # one -hot 时就这样就行 output_size=4
        # self.output = nn.Sigmoid()

    def forward(self, x):
        x_list = []
        temp = 0
        for emb in self.Embedding_list:
            x_list.append(x[:,:,:,temp:temp+emb])
            temp = temp+emb
        embx = []
        for i, emb in enumerate(self.emb):
            embx.append(emb(x_list[i]))
        x = sum(tensor for tensor in embx)
        out = [conv(x) for conv in self.convs]
        out = torch.cat(out, dim=1)
        out = out.view(x.size(0), -1)
        # print(out.shape)
        out = self.fc(out)
        # output = self.output(out)
        return out
    
    
def train(
    net:nn.Module, 
    Train_generator, 
    loss_func,
    optimizer, 
    scheduler, 
    device
):
    net = net.to(device)
    net.train()
    sum_loss = []
    
    Train_generator =  DataLoader(Train_generator.dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
    for x, y in Train_generator:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        scores = net(x)
        loss = loss_func(scores, y)
        sum_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    return np.array(sum_loss).mean()
# def train(net, Train_generator, loss_func, optimizer, scaler, scheduler, device):
#     net = net.to(device)
#     net.train()
#     sum_loss = []
    
#     Train_generator = DataLoader(Train_generator.dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
    
#     for x, y in Train_generator:
#         x = x.to(device)
#         y = y.to(device)
#         optimizer.zero_grad()
        
#         # 使用autocast启用半精度计算
#         with autocast():
#             scores = net(x)
#             loss = loss_func(scores, y)
        
#         # 放大损失值以保持数值稳定性
#         scaler.scale(loss).backward()
        
#         # 应用优化器和梯度缩放
#         scaler.step(optimizer)
#         scaler.update()
        
#         # 清零梯度，为下一次迭代做准备
#         optimizer.zero_grad()
        
#         # 将损失值记录下来
#         sum_loss.append(loss.item())
    
#     # 学习率调度器步骤
#     scheduler.step()
    
#     return np.array(sum_loss).mean()

def evaluate(
    net:nn.Module, 
    Test_generator, 
    loss_func,
    optimizer, 
    scheduler, 
    device
):
    sum_loss = []
    all_errors = []  # 收集所有的误差百分比

    net.eval()
    with torch.no_grad():  # 不计算梯度，减少内存和计算需求
        for x, y in Test_generator:
            x, y = x.to(device), y.to(device)
            scores = net(x)
            loss = loss_func(scores, y)
            sum_loss.append(loss.item())

            # 计算误差百分比
            y_cpu = y.detach().cpu().numpy()
            scores_cpu = scores.detach().cpu().numpy()
            with np.errstate(divide='ignore', invalid='ignore'):
                error_percentage = np.abs((scores_cpu - y_cpu) / y_cpu) * 100
                error_percentage[np.isinf(error_percentage)] = 0  # 处理无穷大
                error_percentage = np.nan_to_num(error_percentage)  # NaN 转为 0

            all_errors.extend(error_percentage)  # 收集所有批次的误差

    average_loss = np.mean(sum_loss)
    average_error_percentage = np.mean(all_errors)  # 计算所有样本的平均误差百分比

    return average_loss, average_error_percentage



def train_multi_epochs(
    path, 
    net:nn.Module, 
    Train_generator, 
    Test_generator,     
    loss_func,
    optimizer, 
    scheduler, 
    epochs, 
    device, 
    information:str, 
    show_train_process=None
):
    best_epoch = 0
    best_test_loss = 9.9e9
    best_net = net.state_dict()
    sum_train_loss, sum_test_loss, sum_per_error = [], [], []
    date0 = time.strftime('%Y-%m-%d %a %H-%M-%S', time.localtime(time.time()))

    with open(path.format(date=date0, information=information), 'w') as log_fin:
        log_fin.write(information + '\n')
        log_fin.write('epoch' + ' ' + 'train_loss' + ' ' + 'test_loss' + ' ' + 'time' + ' ' + 'best_epoch' + '\n')
        t1 = time.time()
        for epoch in range(epochs):
            t0 = time.time()
            train_loss = train(net, Train_generator, loss_func, optimizer, scheduler, device).item()
            test_loss, per_error = evaluate(net, Test_generator, loss_func, optimizer, scheduler, device)#.item()

            sum_train_loss.append(train_loss)
            sum_test_loss.append(test_loss)
            sum_per_error.append(test_loss)
            if epoch == 0 or test_loss < best_test_loss:
                best_epoch = epoch
                best_test_loss = test_loss
                best_net = net.state_dict()

            log_fin.write(str(epoch) + ' ' + str(train_loss) + ' ' + str(test_loss) + ' ' + str(time.time()-t0) + ' ' + str(best_epoch) + ' '+str(per_error) + '\n')
            if show_train_process != None and epoch % show_train_process == 0:
                print('epoch={:>4}, train_loss= {:.4f}, test_loss= {:.4f}, time= {:.2f}sec, best_epoch= {:>4}, per_error={:.4f}'.format(epoch, train_loss, test_loss, time.time()-t1, best_epoch, per_error))
                t1 = time.time()
        
        log_fin.write('\n')
        log_fin.write('best_epoch=' + str(best_epoch) + '\n')
        log_fin.write('best_test_loss=' + str(best_test_loss) + '\n')
        log_fin.write('per_error=' + str(per_error) + '\n')
    return best_test_loss, best_epoch, best_net, sum_train_loss, sum_test_loss, sum_per_error

# def train_multi_epochs(
#     path, 
#     net:nn.Module, 
#     Train_generator, 
#     Test_generator,     
#     loss_func,
#     optimizer, 
#     scheduler,
#     scaler,  # 添加scaler参数
#     epochs, 
#     device, 
#     information:str, 
#     show_train_process=None
# ):
#     best_epoch = 0
#     best_test_loss = 9.9e9
#     best_net = net.state_dict()
#     sum_train_loss, sum_test_loss = [], []
#     date0 = time.strftime('%Y-%m-%d %a %H-%M-%S', time.localtime(time.time()))

#     with open(path.format(date=date0, information=information), 'w') as log_fin:
#         log_fin.write(information + '\n')
#         log_fin.write('epoch' + ' ' + 'train_loss' + ' ' + 'test_loss' + ' ' + 'time' + ' ' + 'best_epoch' + '\n')
#         t1 = time.time()
#         for epoch in range(epochs):
#             t0 = time.time()
#             train_loss = train(net, Train_generator, loss_func, optimizer, scaler, scheduler, device).item()
#             test_loss = evaluate(net, Test_generator, loss_func, optimizer, scheduler, device).item()

#             sum_train_loss.append(train_loss)
#             sum_test_loss.append(test_loss)

#             if epoch == 0 or test_loss < best_test_loss:
#                 best_epoch = epoch
#                 best_test_loss = test_loss
#                 best_net = net.state_dict()

#             log_fin.write(str(epoch) + ' ' + str(train_loss) + ' ' + str(test_loss) + ' ' + str(time.time()-t0) + ' ' + str(best_epoch) + '\n')
#             if show_train_process != None and epoch % show_train_process == 0:
#                 print('epoch={:>4}, train_loss= {:.4f}, test_loss= {:.4f}, time= {:.2f}sec, best_epoch= {:>4}'.format(epoch, train_loss, test_loss, time.time()-t1, best_epoch))
#                 t1 = time.time()
        
#         log_fin.write('\n')
#         log_fin.write('best_epoch=' + str(best_epoch) + '\n')
#         log_fin.write('best_test_loss=' + str(best_test_loss) + '\n')
#     return best_test_loss, best_epoch, best_net, sum_train_loss, sum_test_loss


def singal_train_CNN(i, lr=LEARNING_RATE, ga=0.5, dropout=0.5):
    t1 = time.time()
    # scaler = GradScaler()
    train_iter, test_iter = get_iter(PATH, BATCH_SIZE)
    # test_iter = DataLoader([final_data, final_labels], batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
    # criteon = nn.BCEWithLogitsLoss().to(DEVICE)
    criteon = nn.SmoothL1Loss().to(DEVICE)
    # net = CNN(1, TIME_LENGTH, INPUT_SIZE, 1, 32, [3, 5, 7, 9], dropout)
    # net = CNN(1,TIME_LENGTH//2, 9, [1,6,4,4,3],TIME_LENGTH//2, 8, [9,7,5,3], dropout)
    # net = CNN(1,TIME_LENGTH//2, 25, [1,6,4,4,3], 1, 8, [9,7,5,3], dropout)
    net = CNN(1,BEFORE_LENGTH, 25, [1,6,4,4,3], 1, 8, [9,7,5,3], dropout)
    # net = CNN(1,TIME_LENGTH//2, 25, [3], 1, 8, [9,7,5,3], dropout)
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=0)
    scheduler = MultiStepLR(optimizer, [int(EPOCHS*0.2), int(EPOCHS*0.4)], ga, last_epoch=-1)

    best_test_loss, best_epoch, best_net, multi_train_loss, multi_test_loss,_ = train_multi_epochs(f'./output/{i}', net, train_iter, test_iter, criteon, optimizer, scheduler, EPOCHS, DEVICE, 'Temperature={}'.format(300), show_train_process=10)
    torch.save(best_net, 'best_model_weiyi_Z.pth')
    print('best_test_loss= {:.4f}, best_epoch= {:>4}, time= {:.2f}sec'.format(best_test_loss, best_epoch, time.time()-t1))

    plt.plot(multi_train_loss)
    plt.plot(multi_test_loss)
    plt.savefig(f'./output/{i}.png')
    plt.close()
    return multi_train_loss, multi_test_loss

In [2]:
data = get_original_data(PATH)

In [3]:
data = np.hstack(data[1:])

scaler = StandardScaler()

scaler.fit_transform(data)
mean_scaled = scaler.mean_

var_scaled = scaler.var_

In [8]:
var_scaled**0.5

array([3.89295518e+03, 1.17942775e+00, 5.75954319e-01, 1.69234412e+00,
       1.37599649e+00, 1.34694644e+00, 1.97403935e+00, 8.50576127e+00,
       7.78344116e+00, 8.89985569e+00, 8.88802331e+00, 9.06355493e+00,
       9.09185981e+00, 9.06728313e+00, 6.11193960e+00, 6.83298149e-01,
       6.85452737e-01, 6.81930338e-01])

In [ ]:
model = CNN(1,TIME_LENGTH//2, 25, [1,6,4,4,3], 1, 8, [9,7,5,3], 0.5)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {total_params}')

In [ ]:
# 应该是前10步就能预测了。关键是预测多少步比较好？再算算40步的
# 然后就是流程图
# 然后就是转移矩阵

In [16]:
_,_ = singal_train_CNN('WEIYI_Z')

epoch=   0, train_loss= 0.3622, test_loss= 0.2464, time= 1.21sec, best_epoch=    0, per_error=56.6086
epoch=  10, train_loss= 0.1977, test_loss= 0.1723, time= 12.12sec, best_epoch=   10, per_error=42.4758
epoch=  20, train_loss= 0.1600, test_loss= 0.1353, time= 12.10sec, best_epoch=   20, per_error=36.6089
epoch=  30, train_loss= 0.1365, test_loss= 0.1258, time= 11.99sec, best_epoch=   29, per_error=33.8805
epoch=  40, train_loss= 0.1235, test_loss= 0.1084, time= 12.19sec, best_epoch=   40, per_error=30.3465
epoch=  50, train_loss= 0.1115, test_loss= 0.0990, time= 12.37sec, best_epoch=   49, per_error=27.5770
epoch=  60, train_loss= 0.0996, test_loss= 0.0883, time= 12.24sec, best_epoch=   59, per_error=25.5212
epoch=  70, train_loss= 0.0869, test_loss= 0.0767, time= 12.12sec, best_epoch=   69, per_error=22.9976
epoch=  80, train_loss= 0.0780, test_loss= 0.0679, time= 12.31sec, best_epoch=   80, per_error=20.8679
epoch=  90, train_loss= 0.0699, test_loss= 0.0627, time= 12.28sec, best_ep